In [1]:
import pyomo.environ as pyo
from pyomo.opt.results import SolverStatus
import pandas as pd
import json
import math
import statistics
import random
import re

In [2]:
class Player:
    pref_score = 0
    team = None
    board = None
    req_met = False

    def __init__(self, name, rating, friends, avoid, date, alt, previous_season_alt):
        self.name = name
        self.rating = rating
        self.friends = friends
        self.avoid = avoid
        self.date = date
        self.alt = alt
        self.previous_season_alt = previous_season_alt

    @classmethod
    def player_from_json(cls, player):
        return cls(
            player['name'],
            player['rating'],
            player['friends'],
            player['avoid'],
            player['date_created'],
            player['prefers_alt'],
            player.get('previous_season_alternate', False)
        )

    def __repr__(self):
        return str((self.name, self.board, self.rating, self.req_met))

    def __lt__(self, other):
        return True

    def set_pref_score(self):
        self.pref_score = 0
        for friend in self.friends:
            if friend in self.team.get_boards():
                self.pref_score += 1
            else:
                self.pref_score -= 1
        for avoid in self.avoid:
            if avoid in self.team.get_boards():
                self.pref_score -= 1
        # player with more than 5 choices can be <5 preference even if all teammates are preferred

    def set_req_met(self):
        self.req_met = False
        if not self.friends:
            self.req_met = None
        for friend in self.friends:
            if friend in self.team.get_boards():
                self.req_met = True


class Team:
    def __init__(self, boards):
        self.boards = [None for x in range(boards)]

    def __str__(self):
        return str((self.boards, self.team_pref_score, self.get_mean()))

    def __repr__(self):
        return "Team:{0}".format(id(self))

    def __lt__(self, other):
        return True

    def changeBoard(self, board, new_player):
        # updates the player on a board and updates that player's team attribute
        if self.boards[board]:
            self.boards[board].team = None
        self.boards[board] = new_player
        if new_player.team:
            new_player.team.boards[board] = None
        new_player.team = self

    def get_mean(self, expected_rating=False):
        # expected_rating is an unused parameter in this version.
        # it is used by the tournament.models.Team.get_mean method.
        ratings = [board.rating for board in self.boards]
        mean = sum(ratings) / len(ratings)
        return mean

    def get_boards(self):
        return self.boards

    def get_player(self, board):
        return self.boards[board]

    def set_team_pref_score(self):
        self.team_pref_score = sum([x.pref_score for x in self.boards])
        
def split_into_equal_groups_by_rating(players, group_number):
    players.sort(key=lambda player: player.rating, reverse=True)
    avg = len(players) / float(group_number)
    players_split = []
    last = 0.0
    while round(last) < len(players):
        players_split.append(players[int(round(last)):int(round(last + avg))])
        last += avg
    return players_split


def get_rating_bounds_of_split(split):
    min_ratings = [min([p.rating for p in board]) for board in split]
    max_ratings = [max([p.rating for p in board]) for board in split]
    min_ratings[-1] = 0
    max_ratings[0] = 5000
    return list(zip(min_ratings, max_ratings))

def flatten(lst):
    return [item for sub_lst in lst for item in sub_lst]

In [3]:
# Load data
with open("data.json") as f:
    playerdata = json.load(f)
    
# Parameters
boards = 10
rounds = 8
balance = 0.85
balance_type = 'none' # rating, winexp, or none

In [4]:
players = []
for player in playerdata:
    if player['has_20_games'] and player['in_slack']:
        players.append(Player.player_from_json(player))
    else:
        pass
        # print("{0} skipped".format(player['name']))
players.sort(key=lambda player: player.rating, reverse=True)

# Split into those that want to be alternates vs those that do not.
alternates = [p for p in players if p.alt]
players = [p for p in players if not p.alt]

# splits list of Player objects into near equal lists, sectioned by rating
players_split = split_into_equal_groups_by_rating(players, boards)
team_rating_bounds = get_rating_bounds_of_split(players_split)

num_teams = int(math.ceil((len(players_split[0]) * balance) / 2.0) * 2)

# separate latest joining players into alternate lists as required
for n, board in enumerate(players_split):
    board.sort(key=lambda player: (0 if player.previous_season_alt else 1, player.date))
    alternates.extend(board[num_teams:])
    del board[num_teams:]
    board.sort(key=lambda player: player.rating, reverse=True)

alts_split = split_into_equal_groups_by_rating(alternates, boards)
alt_rating_bounds = get_rating_bounds_of_split(alts_split)

players = flatten(players_split)

for n, board in enumerate(players_split):
    for player in board:
        player.board = n

def convert_name_list(string_of_names, players):
    pattern = r"([^-_a-zA-Z0-9]|^){0}([^-_a-zA-Z0-9]|$)"
    return [player for player in players
            if re.search(pattern.format(player.name), string_of_names, flags=re.I)]

for player in players:
    filtered_players = [p for p in players if p.board != player.board]
    player.friends = convert_name_list(player.friends, filtered_players)
    player.avoid = convert_name_list(player.avoid, filtered_players)

In [5]:
def update_pref(players, teams):  # update preference scores
    for player in players:
        player.set_pref_score()
    for team in teams:
        team.set_team_pref_score()


def update_sort(players, teams):  # based on preference score high to low
    players.sort(key=lambda player: (player.team.team_pref_score, player.pref_score), reverse=False)
    teams.sort(key=lambda team: team.team_pref_score, reverse=False)

In [6]:
# randomly shuffle players
for board in players_split:
    random.shuffle(board)

teams = []
for n in range(num_teams):
    teams.append(Team(boards))
for n, board in enumerate(players_split):
    for team, player in enumerate(board):
        teams[team].changeBoard(n, player)

In [7]:
# Calculated parameters
target_team_rating = statistics.mean([player.rating for player in players])
target_team_winexp = rounds/2

In [67]:
def build_team_optimization_model(players, boards, 
                                  num_teams, target_team_rating, target_team_winexp,
                                  friend_cost=1, avoid_cost=-5,
                                  rating_viol_cost=-1, rating_threshold=10, 
                                  winexp_viol_cost=-50, winexp_threshold=0.05):
    """
    Returns a pyomo model, which can be solved to generate teams 
    (see https://pyomo.readthedocs.io/en/stable/solving_pyomo_models.html).

            Parameters:
                    players (list of objects): List of Player class instances to be placed into teams.
                    boards (int): Number of boards on each team.
                    num_teams (int): Number of teams.
                    target_team_rating (float): The average rating of the player pool.
                    target_team_winexp (float): The number of matches a team should be expected to win.
                    friend_cost (float): The 'happiness' value assigned for one friend pairing on the same team. 
                        Should be a positive number, but this is not enforced.
                    avoid_cost (float): The 'unhappiness' value assigned for a player being placed on the same
                        team as a player they avoided. Should be a negative number, but this is not enforced.
                    rating_viol_cost (float): The 'unhappiness' value assigned for violating the rating
                        threshold by 1 point. Should be a negative number, but this is not enforced.
                    rating_threshold (float): The one-sided size of the window for acceptable team average rating.
                    winexp_viol_cost (float): The 'unhappiness' value assigned for violating the full season 
                        team win expectation by 1 point. Should be a negative number, but this is not enforced.
                    winexp_threshold (float): The one-sided size of the window for acceptable full season team 
                        win expectation.

            Returns:
                    model (object): Instance of pyomo.environ.AbstractModel
    """
    
    # Model
    model = pyo.AbstractModel()

    # Sets
    model.players = pyo.Set(initialize=[player.name for player in players])
    model.boards = pyo.Set(initialize=[board_num for board_num in range(boards)])
    model.teams = pyo.Set(initialize=[team_num for team_num in range(num_teams)])
    model.avoids = pyo.Set(initialize=[(player.name, avoided_player.name) for player in players 
                                       for avoided_player in player.avoid], within=model.players*model.players, dimen=2)
    model.friends = pyo.Set(initialize=[(player.name, friended_player.name) for player in players 
                                        for friended_player in player.friends], within=model.players*model.players, dimen=2)

    # Parameters
    model.param_player_rating = pyo.Param(model.players, 
                                          initialize={player.name:player.rating for player in players}, 
                                          within=pyo.Reals)
    model.param_player_expscore = pyo.Param(model.players, 
                                            initialize={player.name:4.0 for player in players}, 
                                            within=pyo.NonNegativeReals)
    model.param_player_boards = pyo.Param(model.players, 
                                          initialize={player.name:player.board for player in players}, 
                                          within=pyo.NonNegativeIntegers)
    model.param_friend_cost = pyo.Param(initialize=friend_cost, within=pyo.Reals)
    model.param_avoid_cost = pyo.Param(initialize=avoid_cost, within=pyo.Reals)
    model.param_rating_viol_cost = pyo.Param(initialize=rating_viol_cost, within=pyo.Reals)
    model.param_winexp_viol_cost = pyo.Param(initialize=winexp_viol_cost, within=pyo.Reals)
    model.param_rating_threshold = pyo.Param(initialize=rating_threshold, within=pyo.Reals)
    model.param_winexp_threshold = pyo.Param(initialize=winexp_threshold, within=pyo.Reals)

    # Decision variables
    model.dvar_x = pyo.Var(model.players, model.boards, model.teams, within=pyo.Binary, initialize=0)

    # Soft constraint penalty decision variables
    model.pen_friends = pyo.Var(model.friends, model.teams, within=pyo.Binary, initialize=0)
    model.pen_avoids = pyo.Var(model.avoids, model.teams, within=pyo.Binary, initialize=0)
    model.pen_balance_rating = pyo.Var(model.teams, within=pyo.NonNegativeReals, initialize=0)
    model.pen_balance_winexp = pyo.Var(model.teams, within=pyo.NonNegativeReals, initialize=0)

    # Objective
    def objective_function(model):
        return model.param_friend_cost*pyo.summation(model.pen_friends) \
        + model.param_avoid_cost*pyo.summation(model.pen_avoids) \
        + model.param_rating_viol_cost*pyo.summation(model.pen_balance_rating) \
        + model.param_winexp_viol_cost*pyo.summation(model.pen_balance_winexp)
    model.obj = pyo.Objective(rule=objective_function, sense=pyo.maximize)

    # Hard Constraint: Each player must be on one and only one team
    def constraint_one_team(model, p):
        return sum(model.dvar_x[p, b, t] for b in model.boards for t in model.teams) == 1
    model.cons_one_team = pyo.Constraint(model.players, rule=constraint_one_team)
    
    # Hard Constraint: Each team must have exactly the right number of players
    def constraint_right_number_of_players(model, t):
        return sum(model.dvar_x[p, b, t] for p in model.players for b in model.boards) == boards
    model.cons_right_number_of_players = pyo.Constraint(model.teams, rule=constraint_right_number_of_players)

    # Hard Constraint: Each player must be on their own board
    def constraint_own_board(model, p):
        return sum(model.dvar_x[p, model.param_player_boards[p], t] for t in model.teams) == 1
    model.cons_own_board = pyo.Constraint(model.players, rule=constraint_own_board)
    
    # Hard Constraint: One player on each board for each team
    def constraint_one_of_each_board(model, b, t):
        return sum(model.dvar_x[p, b, t] for p in model.players) == 1
    model.cons_one_of_each_board = pyo.Constraint(model.boards, model.teams, rule=constraint_one_of_each_board)

    # Soft Constraint: Teams must be balanced, using rating
    def constraint_team_balance_rating_lb(model, t):
        return sum(model.param_player_rating[p]*model.dvar_x[p, b, t] for p in model.players for b in model.boards) \
        >= boards*target_team_rating - model.param_rating_threshold - model.pen_balance_rating[t]
    model.cons_team_balance_rating_lb = pyo.Constraint(model.teams, rule=constraint_team_balance_rating_lb)

    def constraint_team_balance_rating_ub(model, t):
        return sum(model.param_player_rating[p]*model.dvar_x[p, b, t] for p in model.players for b in model.boards) \
        <= boards*target_team_rating + model.param_rating_threshold + model.pen_balance_rating[t]
    model.cons_team_balance_rating_ub = pyo.Constraint(model.teams, rule=constraint_team_balance_rating_ub)

    # Soft Constraint: Teams must be balanced, using expected wins
    def constraint_winexp_rating_lb(model, t):
        return sum(model.param_player_expscore[p]*model.dvar_x[p, b, t] for p in model.players for b in model.boards) \
        >= target_team_winexp - model.param_winexp_threshold - model.pen_balance_winexp[t]
    model.cons_winexp_rating_lb = pyo.Constraint(model.teams, rule=constraint_winexp_rating_lb)

    def constraint_winexp_rating_ub(model, t):
        return sum(model.param_player_expscore[p]*model.dvar_x[p, b, t] for p in model.players for b in model.boards) \
        <= target_team_winexp + model.param_winexp_threshold + model.pen_balance_winexp[t]
    model.cons_winexp_rating_ub = pyo.Constraint(model.teams, rule=constraint_winexp_rating_ub)

    # Soft Constraint: Players should not be placed on the same team as a player on their avoid list
    def constraint_avoid_list(model, p1, p2, t):
        return 2*model.pen_avoids[(p1, p2), t] \
        <= sum(model.dvar_x[p1, b, t]+model.dvar_x[p2, b, t] for b in model.boards)
    model.cons_avoid_list = pyo.Constraint(model.avoids, model.teams, rule=constraint_avoid_list)

    # Soft Constraint: Players should be placed with their friends
    def constraint_friend_list(model, p1, p2, t):
        return 2*model.pen_friends[(p1, p2), t] \
        <= sum(model.dvar_x[p1, b, t] + model.dvar_x[p2, b, t] for b in model.boards)
    model.cons_friend_list = pyo.Constraint(model.friends, model.teams, rule=constraint_friend_list)
    
    return model

In [68]:
model = build_team_optimization_model(players, boards, num_teams, target_team_rating, target_team_winexp)
instance = model.create_instance()

# Simple model - deactivate all team balance constraints
if balance_type == 'none':
    instance.cons_team_balance_rating_lb.deactivate()
    instance.cons_team_balance_rating_ub.deactivate()
    instance.cons_winexp_rating_lb.deactivate()
    instance.cons_winexp_rating_ub.deactivate()
elif balance_type == 'rating':
    instance.cons_winexp_rating_lb.deactivate()
    instance.cons_winexp_rating_ub.deactivate()
elif balance_type == 'winexp':
    instance.cons_team_balance_rating_lb.deactivate()
    instance.cons_team_balance_rating_ub.deactivate()

In [69]:
opt = pyo.SolverFactory('glpk')
opt.options['tmlim'] = 600
result = opt.solve(instance, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 600 --write C:\Users\KEVINC~1\AppData\Local\Temp\tmpj77nb3cq.glpk.raw
 --wglp C:\Users\KEVINC~1\AppData\Local\Temp\tmp9en_aip6.glpk.glp --cpxlp
 C:\Users\KEVINC~1\AppData\Local\Temp\tmpsho8d2tf.pyomo.lp
Reading problem data from 'C:\Users\KEVINC~1\AppData\Local\Temp\tmpsho8d2tf.pyomo.lp'...
C:\Users\KEVINC~1\AppData\Local\Temp\tmpsho8d2tf.pyomo.lp:703893: warning: lower bound of variable 'x1' redefined
C:\Users\KEVINC~1\AppData\Local\Temp\tmpsho8d2tf.pyomo.lp:703893: warning: upper bound of variable 'x1' redefined
1241 rows, 182081 columns, 496001 non-zeros
182000 integer variables, all of which are binary
885893 lines were read
Writing problem data to 'C:\Users\KEVINC~1\AppData\Local\Temp\tmp9en_aip6.glpk.glp'...
702728 lines were written
GLPK Integer Optimizer, v4.65
1241 rows, 182081 columns, 496001 non-zeros
182000 integer variables, all of which are binary
Preprocessing...
1240 rows, 160000 colu

In [70]:
result.Solver.Status = SolverStatus.warning
instance.solutions.load_from(result)

    model.name="unknown";
      - termination condition: optimal
      - message from solver: <undefined>


In [71]:
result

{'Problem': [{'Name': 'unknown', 'Lower bound': 20000.0, 'Upper bound': 20000.0, 'Number of objectives': 1, 'Number of constraints': 1241, 'Number of variables': 182081, 'Number of nonzeros': 496001, 'Sense': 'maximize'}], 'Solver': [{'Status': 'warning', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 3.0224506855010986}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [72]:
instance.obj.display()

obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 20000.0


In [73]:
team_assignments = []
for v in instance.component_objects(pyo.Var, active=True):
    print(v.name)
    if v.name == 'dvar_x':
        for index in v:
            if pyo.value(v[index]) > 0:
                team_assignments.append(index)
    for index in v:
        if pyo.value(v[index]) > 0:
            print ("   ",index, pyo.value(v[index]))
a = pd.DataFrame(team_assignments, columns=['name', 'board', 'team'])

dvar_x
    ('lovlas', 0, 30) 1.0
    ('Sigrud', 0, 18) 1.0
    ('FrugoFruit90', 0, 7) 1.0
    ('god666', 0, 27) 1.0
    ('Fearlessking64', 0, 4) 1.0
    ('IsaVulpes', 0, 9) 1.0
    ('skyfaller1', 0, 34) 1.0
    ('makeareason', 0, 32) 1.0
    ('MoistvonLipwig', 0, 15) 1.0
    ('Ajolote4', 0, 0) 1.0
    ('victormanolo', 0, 37) 1.0
    ('cantosh', 0, 23) 1.0
    ('okoros', 0, 33) 1.0
    ('HNorbi96', 0, 8) 1.0
    ('LostEarthworm', 0, 13) 1.0
    ('jessehf', 0, 28) 1.0
    ('Bjorgvin1', 0, 1) 1.0
    ('LandyPJY', 0, 12) 1.0
    ('FlokiTheCat', 0, 6) 1.0
    ('lundiz', 0, 31) 1.0
    ('Kjar', 0, 11) 1.0
    ('FirstnameLastname', 0, 5) 1.0
    ('yeyq', 0, 39) 1.0
    ('Razorneck', 0, 16) 1.0
    ('toddletales', 0, 36) 1.0
    ('Trexchess', 0, 22) 1.0
    ('eie24', 0, 25) 1.0
    ('Champignon', 0, 2) 1.0
    ('jumpingspidersrcute', 0, 29) 1.0
    ('Mariuseg', 0, 14) 1.0
    ('JunkyKong', 0, 10) 1.0
    ('tmoila', 0, 35) 1.0
    ('SycoraxCirce', 0, 20) 1.0
    ('david252525', 0, 24) 1.0
    (

    ('traful', 9, 38) 1.0
    ('dakillian', 9, 16) 1.0
    ('Dingoe12', 9, 3) 1.0
    ('lvdr', 9, 28) 1.0
    ('saintex', 9, 35) 1.0
    ('mojomoe', 9, 31) 1.0
    ('simopippa', 9, 36) 1.0
    ('Tomato_Xplosion', 9, 9) 1.0
    ('sutoneIGT', 9, 37) 1.0
    ('ipr', 9, 24) 1.0
    ('DrearyHelix', 9, 4) 1.0
    ('rsava', 9, 34) 1.0
    ('jamzskee', 9, 25) 1.0
    ('rinrinrin', 9, 32) 1.0
    ('dama_x_rey', 9, 17) 1.0
    ('QHov', 9, 8) 1.0
    ('mmorrisohio', 9, 30) 1.0
    ('asn8745', 9, 13) 1.0
    ('Cornelius25', 9, 2) 1.0
    ('kinozarco', 9, 27) 1.0
    ('houcharlie', 9, 23) 1.0
    ('greener255', 9, 22) 1.0
    ('caodanny', 9, 14) 1.0
    ('Ferchosalgado', 9, 6) 1.0
    ('cinqued', 9, 15) 1.0
    ('espnjosh', 9, 20) 1.0
    ('Eshpy', 9, 5) 1.0
    ('alex123a', 9, 12) 1.0
    ('VeeDeltaVee', 9, 10) 1.0
    ('jsettle1', 9, 26) 1.0
    ('CalculatedChess', 9, 0) 1.0
    ('gotthatplymouthrock', 9, 21) 1.0
    ('z0x', 9, 39) 1.0
    ('ChewieMarbles', 9, 1) 1.0
    ('rookontherim', 9, 33) 1

    ('victormanolo', 'timothyha', 1) 1.0
    ('victormanolo', 'timothyha', 2) 1.0
    ('victormanolo', 'timothyha', 3) 1.0
    ('victormanolo', 'timothyha', 4) 1.0
    ('victormanolo', 'timothyha', 5) 1.0
    ('victormanolo', 'timothyha', 6) 1.0
    ('victormanolo', 'timothyha', 7) 1.0
    ('victormanolo', 'timothyha', 8) 1.0
    ('victormanolo', 'timothyha', 9) 1.0
    ('victormanolo', 'timothyha', 10) 1.0
    ('victormanolo', 'timothyha', 11) 1.0
    ('victormanolo', 'timothyha', 12) 1.0
    ('victormanolo', 'timothyha', 13) 1.0
    ('victormanolo', 'timothyha', 14) 1.0
    ('victormanolo', 'timothyha', 15) 1.0
    ('victormanolo', 'timothyha', 16) 1.0
    ('victormanolo', 'timothyha', 17) 1.0
    ('victormanolo', 'timothyha', 18) 1.0
    ('victormanolo', 'timothyha', 19) 1.0
    ('victormanolo', 'timothyha', 20) 1.0
    ('victormanolo', 'timothyha', 21) 1.0
    ('victormanolo', 'timothyha', 22) 1.0
    ('victormanolo', 'timothyha', 23) 1.0
    ('victormanolo', 'timothyha', 24) 1.0
 

    ('LostEarthworm', 'caodanny', 14) 1.0
    ('LostEarthworm', 'caodanny', 15) 1.0
    ('LostEarthworm', 'caodanny', 16) 1.0
    ('LostEarthworm', 'caodanny', 17) 1.0
    ('LostEarthworm', 'caodanny', 18) 1.0
    ('LostEarthworm', 'caodanny', 19) 1.0
    ('LostEarthworm', 'caodanny', 20) 1.0
    ('LostEarthworm', 'caodanny', 21) 1.0
    ('LostEarthworm', 'caodanny', 22) 1.0
    ('LostEarthworm', 'caodanny', 23) 1.0
    ('LostEarthworm', 'caodanny', 24) 1.0
    ('LostEarthworm', 'caodanny', 25) 1.0
    ('LostEarthworm', 'caodanny', 26) 1.0
    ('LostEarthworm', 'caodanny', 27) 1.0
    ('LostEarthworm', 'caodanny', 28) 1.0
    ('LostEarthworm', 'caodanny', 29) 1.0
    ('LostEarthworm', 'caodanny', 30) 1.0
    ('LostEarthworm', 'caodanny', 31) 1.0
    ('LostEarthworm', 'caodanny', 32) 1.0
    ('LostEarthworm', 'caodanny', 33) 1.0
    ('LostEarthworm', 'caodanny', 34) 1.0
    ('LostEarthworm', 'caodanny', 35) 1.0
    ('LostEarthworm', 'caodanny', 36) 1.0
    ('LostEarthworm', 'caodanny', 

    ('Vlad_G92', 'SedNomini', 34) 1.0
    ('Vlad_G92', 'SedNomini', 35) 1.0
    ('Vlad_G92', 'SedNomini', 36) 1.0
    ('Vlad_G92', 'SedNomini', 37) 1.0
    ('Vlad_G92', 'SedNomini', 38) 1.0
    ('Vlad_G92', 'SedNomini', 39) 1.0
    ('Grzybica', 'SedNomini', 0) 1.0
    ('Grzybica', 'SedNomini', 1) 1.0
    ('Grzybica', 'SedNomini', 2) 1.0
    ('Grzybica', 'SedNomini', 3) 1.0
    ('Grzybica', 'SedNomini', 4) 1.0
    ('Grzybica', 'SedNomini', 5) 1.0
    ('Grzybica', 'SedNomini', 6) 1.0
    ('Grzybica', 'SedNomini', 7) 1.0
    ('Grzybica', 'SedNomini', 8) 1.0
    ('Grzybica', 'SedNomini', 9) 1.0
    ('Grzybica', 'SedNomini', 10) 1.0
    ('Grzybica', 'SedNomini', 11) 1.0
    ('Grzybica', 'SedNomini', 12) 1.0
    ('Grzybica', 'SedNomini', 13) 1.0
    ('Grzybica', 'SedNomini', 14) 1.0
    ('Grzybica', 'SedNomini', 15) 1.0
    ('Grzybica', 'SedNomini', 16) 1.0
    ('Grzybica', 'SedNomini', 17) 1.0
    ('Grzybica', 'SedNomini', 18) 1.0
    ('Grzybica', 'SedNomini', 19) 1.0
    ('Grzybica', 'SedN

...

    ('lvdr', 'VicPez', 18) 1.0
    ('lvdr', 'VicPez', 19) 1.0
    ('lvdr', 'VicPez', 20) 1.0
    ('lvdr', 'VicPez', 21) 1.0
    ('lvdr', 'VicPez', 22) 1.0
    ('lvdr', 'VicPez', 23) 1.0
    ('lvdr', 'VicPez', 24) 1.0
    ('lvdr', 'VicPez', 25) 1.0
    ('lvdr', 'VicPez', 26) 1.0
    ('lvdr', 'VicPez', 27) 1.0
    ('lvdr', 'VicPez', 28) 1.0
    ('lvdr', 'VicPez', 29) 1.0
    ('lvdr', 'VicPez', 30) 1.0
    ('lvdr', 'VicPez', 31) 1.0
    ('lvdr', 'VicPez', 32) 1.0
    ('lvdr', 'VicPez', 33) 1.0
    ('lvdr', 'VicPez', 34) 1.0
    ('lvdr', 'VicPez', 35) 1.0
    ('lvdr', 'VicPez', 36) 1.0
    ('lvdr', 'VicPez', 37) 1.0
    ('lvdr', 'VicPez', 38) 1.0
    ('lvdr', 'VicPez', 39) 1.0
    ('lvdr', 'nsmiller7', 0) 1.0
    ('lvdr', 'nsmiller7', 1) 1.0
    ('lvdr', 'nsmiller7', 2) 1.0
    ('lvdr', 'nsmiller7', 3) 1.0
    ('lvdr', 'nsmiller7', 4) 1.0
    ('lvdr', 'nsmiller7', 5) 1.0
    ('lvdr', 'nsmiller7', 6) 1.0
    ('lvdr', 'nsmiller7', 7) 1.0
    ('lvdr', 'nsmiller7', 8) 1.0
    ('lvdr', 'nsmille

    ('mojomoe', 'gavalanche20', 32) 1.0
    ('mojomoe', 'gavalanche20', 33) 1.0
    ('mojomoe', 'gavalanche20', 34) 1.0
    ('mojomoe', 'gavalanche20', 35) 1.0
    ('mojomoe', 'gavalanche20', 36) 1.0
    ('mojomoe', 'gavalanche20', 37) 1.0
    ('mojomoe', 'gavalanche20', 38) 1.0
    ('mojomoe', 'gavalanche20', 39) 1.0
    ('mojomoe', 'loungelizard', 0) 1.0
    ('mojomoe', 'loungelizard', 1) 1.0
    ('mojomoe', 'loungelizard', 2) 1.0
    ('mojomoe', 'loungelizard', 3) 1.0
    ('mojomoe', 'loungelizard', 4) 1.0
    ('mojomoe', 'loungelizard', 5) 1.0
    ('mojomoe', 'loungelizard', 6) 1.0
    ('mojomoe', 'loungelizard', 7) 1.0
    ('mojomoe', 'loungelizard', 8) 1.0
    ('mojomoe', 'loungelizard', 9) 1.0
    ('mojomoe', 'loungelizard', 10) 1.0
    ('mojomoe', 'loungelizard', 11) 1.0
    ('mojomoe', 'loungelizard', 12) 1.0
    ('mojomoe', 'loungelizard', 13) 1.0
    ('mojomoe', 'loungelizard', 14) 1.0
    ('mojomoe', 'loungelizard', 15) 1.0
    ('mojomoe', 'loungelizard', 16) 1.0
    ('mojo

    ('Ferchosalgado', 'diegohdragon', 11) 1.0
    ('Ferchosalgado', 'diegohdragon', 12) 1.0
    ('Ferchosalgado', 'diegohdragon', 13) 1.0
    ('Ferchosalgado', 'diegohdragon', 14) 1.0
    ('Ferchosalgado', 'diegohdragon', 15) 1.0
    ('Ferchosalgado', 'diegohdragon', 16) 1.0
    ('Ferchosalgado', 'diegohdragon', 17) 1.0
    ('Ferchosalgado', 'diegohdragon', 18) 1.0
    ('Ferchosalgado', 'diegohdragon', 19) 1.0
    ('Ferchosalgado', 'diegohdragon', 20) 1.0
    ('Ferchosalgado', 'diegohdragon', 21) 1.0
    ('Ferchosalgado', 'diegohdragon', 22) 1.0
    ('Ferchosalgado', 'diegohdragon', 23) 1.0
    ('Ferchosalgado', 'diegohdragon', 24) 1.0
    ('Ferchosalgado', 'diegohdragon', 25) 1.0
    ('Ferchosalgado', 'diegohdragon', 26) 1.0
    ('Ferchosalgado', 'diegohdragon', 27) 1.0
    ('Ferchosalgado', 'diegohdragon', 28) 1.0
    ('Ferchosalgado', 'diegohdragon', 29) 1.0
    ('Ferchosalgado', 'diegohdragon', 30) 1.0
    ('Ferchosalgado', 'diegohdragon', 31) 1.0
    ('Ferchosalgado', 'diegohdrago

    ('Eshpy', 'Chewbacca_Defense', 25) 1.0
    ('Eshpy', 'Chewbacca_Defense', 26) 1.0
    ('Eshpy', 'Chewbacca_Defense', 27) 1.0
    ('Eshpy', 'Chewbacca_Defense', 28) 1.0
    ('Eshpy', 'Chewbacca_Defense', 29) 1.0
    ('Eshpy', 'Chewbacca_Defense', 30) 1.0
    ('Eshpy', 'Chewbacca_Defense', 31) 1.0
    ('Eshpy', 'Chewbacca_Defense', 32) 1.0
    ('Eshpy', 'Chewbacca_Defense', 33) 1.0
    ('Eshpy', 'Chewbacca_Defense', 34) 1.0
    ('Eshpy', 'Chewbacca_Defense', 35) 1.0
    ('Eshpy', 'Chewbacca_Defense', 36) 1.0
    ('Eshpy', 'Chewbacca_Defense', 37) 1.0
    ('Eshpy', 'Chewbacca_Defense', 38) 1.0
    ('Eshpy', 'Chewbacca_Defense', 39) 1.0
    ('Eshpy', 'Lupo_Jones', 0) 1.0
    ('Eshpy', 'Lupo_Jones', 1) 1.0
    ('Eshpy', 'Lupo_Jones', 2) 1.0
    ('Eshpy', 'Lupo_Jones', 3) 1.0
    ('Eshpy', 'Lupo_Jones', 4) 1.0
    ('Eshpy', 'Lupo_Jones', 5) 1.0
    ('Eshpy', 'Lupo_Jones', 6) 1.0
    ('Eshpy', 'Lupo_Jones', 7) 1.0
    ('Eshpy', 'Lupo_Jones', 8) 1.0
    ('Eshpy', 'Lupo_Jones', 9) 1.0
    (

    ('electricfalcon', 'Sesquipedalism', 4) 1.0
    ('electricfalcon', 'Sesquipedalism', 5) 1.0
    ('electricfalcon', 'Sesquipedalism', 6) 1.0
    ('electricfalcon', 'Sesquipedalism', 7) 1.0
    ('electricfalcon', 'Sesquipedalism', 8) 1.0
    ('electricfalcon', 'Sesquipedalism', 9) 1.0
    ('electricfalcon', 'Sesquipedalism', 10) 1.0
    ('electricfalcon', 'Sesquipedalism', 11) 1.0
    ('electricfalcon', 'Sesquipedalism', 12) 1.0
    ('electricfalcon', 'Sesquipedalism', 13) 1.0
    ('electricfalcon', 'Sesquipedalism', 14) 1.0
    ('electricfalcon', 'Sesquipedalism', 15) 1.0
    ('electricfalcon', 'Sesquipedalism', 16) 1.0
    ('electricfalcon', 'Sesquipedalism', 17) 1.0
    ('electricfalcon', 'Sesquipedalism', 18) 1.0
    ('electricfalcon', 'Sesquipedalism', 19) 1.0
    ('electricfalcon', 'Sesquipedalism', 20) 1.0
    ('electricfalcon', 'Sesquipedalism', 21) 1.0
    ('electricfalcon', 'Sesquipedalism', 22) 1.0
    ('electricfalcon', 'Sesquipedalism', 23) 1.0
    ('electricfalcon', 'Se

In [74]:
a = a.merge(pd.DataFrame(playerdata)[['name', 'rating']])
a[['team', 'rating']].groupby('team', as_index=False).mean()

,team,rating
0,0,1862.4
1,1,1852.3
2,2,1862.5
3,3,1861.1
4,4,1873.7
5,5,1852.0
6,6,1852.0
7,7,1850.3
8,8,1871.2
9,9,1883.4
